##Preparation before filtering loop

In [ ]:
#imports
import json
import requests
import pandas as pd

In [ ]:
# giving access to google drive to access json files
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
# directory the json file is in (change it based on the directory you put the dataset in)
data_path = 'drive/My Drive/projects/data/bxsci-cord-19/'

# reading the meta file into a pandas dataframe so that we can access the names of every json file and their directory
meta = pd.read_csv(data_path+'CORD-19-research-challenge/metadata.csv') 

k = meta['sha'].notna() # assigning variable with all the articles that have text in it (some articles dont have text)
meta = meta[k].reset_index() # clearing out all the articles in the metadata file that dont have anything (labled Na)

k = meta['has_full_text'] # assigning variable with all the articles that have full text avalible in the dataset
meta = meta[k].reset_index() # clearing out all the aricles that dont have full text avalible
meta

,level_0,index,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file,url
0,0,0,vho70jcx,f056da9c64fbf00a4645ae326e8a4339d015d155,biorxiv,SIANN: Strain Identification by Alignment to N...,10.1101/001727,NaN,NaN,biorxiv,Next-generation sequencing is increasingly bei...,2014-01-10,Samuel Minot; Stephen D Turner; Krista L Ternu...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/001727
1,1,1,i9tbix2v,daf32e013d325a6feb80e83d15aabc64a48fae33,biorxiv,Spatial epidemiology of networked metapopulati...,10.1101/003889,NaN,NaN,biorxiv,An emerging disease is one infectious epidemic...,2014-06-04,Lin WANG; Xiang Li,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/003889
2,2,2,62gfisc6,f33c6d94b0efaa198f8f3f20e644625fa3fe10d2,biorxiv,Sequencing of the human IG light chain loci fr...,10.1101/006866,NaN,NaN,biorxiv,Germline variation at immunoglobulin gene (IG)...,2014-07-03,Corey T Watson; Karyn Meltz Steinberg; Tina A ...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/006866
3,3,3,058r9486,4da8a87e614373d56070ed272487451266dce919,biorxiv,Bayesian mixture analysis for metagenomic comm...,10.1101/007476,NaN,NaN,biorxiv,Deep sequencing of clinical samples is now an ...,2014-07-25,Sofia Morfopoulou; Vincent Plagnol,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/007476
4,4,4,wich35l7,eccef80cfbe078235df22398f195d5db462d8000,biorxiv,Mapping a viral phylogeny onto outbreak trees ...,10.1101/010389,NaN,NaN,biorxiv,Developing methods to reconstruct transmission...,2014-11-11,Stephen P Velsko; Jonathan E Allen,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/010389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31748,31748,45767,b7e9grj0,889ba9338ea71cd42c3bc675db30a1928d487f43; d38e...,Elsevier,Relative immunogenicity and protection potenti...,10.1016/j.vaccine.2008.01.024,PMC2288748,18291562.0,els-covid,"Summary Yersinia Pestis outer proteins, plasmi...",2008-03-20,"Wang, Shixia; Joshi, Swati; Mboudjeka, Innocen...",Vaccine,NaN,NaN,True,custom_license,https://doi.org/10.1016/j.vaccine.2008.01.024
31749,31749,45768,6b1y7yxg,f81692543d3e35858911cea48c298bfa23b20bc6,Elsevier,Quality of life and psychological status in su...,10.1016/j.jpsychores.2005.08.020,PMC7094294,16650592.0,els-covid,Abstract Background Little is known about the ...,2006-05-31,"Kwek, Seow-Khee; Chew, Wuen-Ming; Ong, Kian-Ch...",Journal of Psychosomatic Research,NaN,NaN,True,custom_license,https://doi.org/10.1016/j.jpsychores.2005.08.020
31750,31750,45769,4360s2yu,289deae0b2050aa259a05ba84565a4df82fa099a,Elsevier,Personal Protective Equipment: Protecting Heal...,10.1016/j.clinthera.2015.07.007,PMC4661082,26452427.0,els-covid,Abstract Purpose The recent Ebola epidemic tha...,2015-11-01,"Fischer, William A.; Weber, David J.; Wohl, Da...",Clinical Therapeutics,NaN,NaN,True,custom_license,https://doi.org/10.1016/j.clinthera.2015.07.007
31751,31751,45770,66jumbir,21a4369f83891bf6975dd916c0aa495d5df8709e,Elsevier,Viruses and asthma,10.1016/j.bbagen.2011.01.012,PMC3130828,21291960.0,els-covid,Abstract Background Viral respiratory infectio...,2011-11-30,"Dulek, Daniel E.; Peebles, R. Stokes",Biochimica et Biophysica Acta (BBA) - General ...,NaN,NaN,True,custom_license,https://doi.org/10.1016/j.bbagen.2011.01.012


For some reason some rows in meta dataframe has more than one ID when we are calling for the paper ID. So this cell below splits the ids with more than one in it, into each row seperatly so that we can call on each ID seperatly, access the directory of the paper and the text we want to extract from it.

In [ ]:
meta['sha'] = meta['sha'].str.split(';')
meta = meta.explode('sha')
meta = meta.reset_index(drop = True) # also resetting the index so that the loop doesnt skip over or confuse some rows during the loop
meta

,level_0,index,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file,url
0,0,0,vho70jcx,f056da9c64fbf00a4645ae326e8a4339d015d155,biorxiv,SIANN: Strain Identification by Alignment to N...,10.1101/001727,NaN,NaN,biorxiv,Next-generation sequencing is increasingly bei...,2014-01-10,Samuel Minot; Stephen D Turner; Krista L Ternu...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/001727
1,1,1,i9tbix2v,daf32e013d325a6feb80e83d15aabc64a48fae33,biorxiv,Spatial epidemiology of networked metapopulati...,10.1101/003889,NaN,NaN,biorxiv,An emerging disease is one infectious epidemic...,2014-06-04,Lin WANG; Xiang Li,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/003889
2,2,2,62gfisc6,f33c6d94b0efaa198f8f3f20e644625fa3fe10d2,biorxiv,Sequencing of the human IG light chain loci fr...,10.1101/006866,NaN,NaN,biorxiv,Germline variation at immunoglobulin gene (IG)...,2014-07-03,Corey T Watson; Karyn Meltz Steinberg; Tina A ...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/006866
3,3,3,058r9486,4da8a87e614373d56070ed272487451266dce919,biorxiv,Bayesian mixture analysis for metagenomic comm...,10.1101/007476,NaN,NaN,biorxiv,Deep sequencing of clinical samples is now an ...,2014-07-25,Sofia Morfopoulou; Vincent Plagnol,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/007476
4,4,4,wich35l7,eccef80cfbe078235df22398f195d5db462d8000,biorxiv,Mapping a viral phylogeny onto outbreak trees ...,10.1101/010389,NaN,NaN,biorxiv,Developing methods to reconstruct transmission...,2014-11-11,Stephen P Velsko; Jonathan E Allen,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/010389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33417,31749,45768,6b1y7yxg,f81692543d3e35858911cea48c298bfa23b20bc6,Elsevier,Quality of life and psychological status in su...,10.1016/j.jpsychores.2005.08.020,PMC7094294,16650592.0,els-covid,Abstract Background Little is known about the ...,2006-05-31,"Kwek, Seow-Khee; Chew, Wuen-Ming; Ong, Kian-Ch...",Journal of Psychosomatic Research,NaN,NaN,True,custom_license,https://doi.org/10.1016/j.jpsychores.2005.08.020
33418,31750,45769,4360s2yu,289deae0b2050aa259a05ba84565a4df82fa099a,Elsevier,Personal Protective Equipment: Protecting Heal...,10.1016/j.clinthera.2015.07.007,PMC4661082,26452427.0,els-covid,Abstract Purpose The recent Ebola epidemic tha...,2015-11-01,"Fischer, William A.; Weber, David J.; Wohl, Da...",Clinical Therapeutics,NaN,NaN,True,custom_license,https://doi.org/10.1016/j.clinthera.2015.07.007
33419,31751,45770,66jumbir,21a4369f83891bf6975dd916c0aa495d5df8709e,Elsevier,Viruses and asthma,10.1016/j.bbagen.2011.01.012,PMC3130828,21291960.0,els-covid,Abstract Background Viral respiratory infectio...,2011-11-30,"Dulek, Daniel E.; Peebles, R. Stokes",Biochimica et Biophysica Acta (BBA) - General ...,NaN,NaN,True,custom_license,https://doi.org/10.1016/j.bbagen.2011.01.012
33420,31752,45773,28vx9w58,3369a14e1d116943f48b3a33597796c9802de279,PMC,Searching for animal models and potential targ...,10.1016/j.onehlt.2017.03.001,PMC5454147,28616501.0,cc-by-nc-nd,Emerging and re-emerging pathogens represent a...,2017-03-03,"Vergara-Alert, Júlia; Vidal, Enric; Bensaid, A...",One Health,NaN,NaN,True,noncomm_use_subset,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...


##Main text filtering loop

In [ ]:
Cord_19_dataframe = pd.DataFrame(columns=['article','part','text']) # creating the datafram we will put all the data in

# main filtering loop:
for i in meta.index:
  skip = False

  a_paper_info = meta.iloc[i] # choosing the information of a single article to focus on (a single row on the meta dataframe)
  #making the string of the path of the article we are using below:
  paper_path = data_path+'CORD-19-research-challenge/'+a_paper_info['full_text_file']+'/'+a_paper_info['full_text_file']+'/'+a_paper_info['sha']+'.json'
  article = a_paper_info['sha'] #assigning the name of the article to the the variable article

# sometimes the article in the meta dataframe doest exsist yet, this code ensures the whole loop keeps going
  try:
    with open(paper_path,'r') as opened_json:
      a_json = json.load(opened_json)
  except:
    print(i,article)
    skip = True

# this part of the loop appends all the text we want from each part of the paper into the main dataframe
  if not skip: # if skip = true the loop skips over this so the loop keeps going
    if len(a_json['abstract'])>0: # sometimes the sections of the article are empty, this makes sure the loop doesnt stop because of that
      abstract_df = pd.DataFrame({'article':article,'part':'abstract','text':pd.DataFrame.from_dict(a_json['abstract'])['text']})
      Cord_19_dataframe = Cord_19_dataframe.append(abstract_df, ignore_index = True) 
    if len(a_json['body_text'])>0:
      body_df = pd.DataFrame({'article':article,'part':'body','text':pd.DataFrame.from_dict(a_json['body_text'])['text']})
      Cord_19_dataframe = Cord_19_dataframe.append(body_df, ignore_index = True)
    if len(a_json['bib_entries'])>0:  
      bib_df = pd.DataFrame({'article':article,'part':'bib','text':pd.DataFrame.from_dict(a_json['bib_entries']).loc['title']}).reset_index(drop=True)
      Cord_19_dataframe = Cord_19_dataframe.append(bib_df, ignore_index = True)
    if len(a_json['ref_entries'])>0: 
      ref_df = pd.DataFrame({'article':article,'part':'ref','text':pd.DataFrame.from_dict(a_json['ref_entries']).loc['text']}).reset_index(drop=True)
      Cord_19_dataframe = Cord_19_dataframe.append(ref_df, ignore_index = True) 

  if i%250==0: # prints i every 250 iterations
    print(i)

# the code below saves the dataframe into a pkl file on the google drive
# this makes it so that we dont have to run this loop again everytime we need to use the dataset
Cord_19_dataframe.to_pickle(data_path+'Cord_19_filtered.pkl') 

0
250
500
750
1000
1112 1e1286db212100993d03cc22374b624f7caee956
1113 8ae137c8da1607b3a8e4c946c07ca8bda67f88ac
1138  4a228865af2c19adf9386a5b04cca3ceb3c8683d
1139  cdb29ec7a9029d22f6fbf7ee04543819591acdc2
1219  752693d2137be042f6d7e42e1aa034f5bfb95d9f
1250
1322  6fe7dbbe77090cf3a1ac5ce68b0aca877c08dd71
1323  84ad793db6fd5878929791f4a2f50943881206f1
1324  f6686ac18c05ec72b33fbed5485fc922531d2529
1325  76481505399740984fe9bf11fdfec984fbf11e32
1381  d4f0247db5e916c20eae3f6d772e8572eb828236
1384  ad9ac0ac5e7da097253fd545b56e2b15ee9de34f
1386  8b39433dd865c0f71c7b2f333e1f506b73d722f1
1461  98d0756af42f78bb74a9df2846729c36ba2f05ad
1493  c235de11eceb5f9f55e0b361818e959d4482b621
1495  8d14b700a065187eb4d8b01e0e9b6e3e37e5d09b
1496  898be097851a56d857d6cdb8ccbbdd3666eb4963
1500
1607  99da41d8c3a3aebdaf5d889c3958181ae52ff6a9
1608  8a18be6bbec92094276f843c39fa9ea509e20215
1624  26b43aa70c7a8314456956b5ada051dc6807dc56
1641  cc724dd07268172a4d5b9d83df29fe32757dee98
1715  eed4a68d4e44f9887b3219ad581

## For future use of the filtered dataset

In [ ]:
# This will read the pkl file we saved in the last cell back into the notebook into the variable Cord_19_dataframe
Cord_19_dataframe = pd.read_pickle(data_path+'Cord_19_filtered.pkl')

## Json structure

(text we are want and are going to filter out is labled as such)

'paper_id'

'metadata'
- 'title'
- 'authors'
  - 0
    - 'first'
    - 'middle'
      - 0
      - 1
      - (whatever number)
    - 'last'
    - 'suffix'
    - 'affiliation'
      - 'laboratory'
      - 'institution'
      - 'location' (information present in here changes a lot, sometimes one of these peices of info are there, sometimes there is nothing. There may also be more info present than listed here but its not relevant)
        - 'postCode'
        - 'addrLine'
        - 'settlement'
        - 'region'
        - 'country'
    - 'email'
  - 1
  - (whatever number)

'abstract'
- 0
  - 'cite_spans'
      - 0
        - 'start'
        - 'end'
        - 'text' (not relevant)
        - 'ref_id'
      - 1
      - (whatever number)
  - 'ref_spans'
    - 0
      - 'start'
      - 'end'
      - 'text' (not relevant)
      - 'ref_id'
    - 1
    - (whatever number)
  - 'section'
  - 'text' (to be extracted)
- 1
- (whatever number)

'body_text'
- 0
  - 'text' (to be extracted)
  - 'cite_spans'
    - 0
      - 'start'
      - 'end'
      - 'text' (not relevant)
      - 'ref_id'
    - 1
    - (whatever number)
  - 'ref_spans'
    - 0
      - 'start'
      - 'end'
      - 'text' (not relevant)
      - 'ref_id'
    - 1
    - (whatever number)
  - 'section'
- 1
- (whatever number)

'bib_entries' (sometimes the number after BIBREF is off like from BIBREF1 to BIBREF3 with no BIBREF2)
- 'BIBREF0'
  - 'title' (to be extracted)
  - 'authors'
    - 0
      - 'first'
      - 'middle'
        - 0
        - 1
        - (whatever number)
      - 'last'
      - 'suffix'
    - 1
    - (whatever number)
  - 'year'
  - 'venue'
  - 'volume'
  - 'issn'
  - 'pages'
  - 'other_ids'
    - 'DOI'
      - 0
      - 1 
      - (whatever number)
- 'BIBREF1'
- 'BIBREF (whatever number)'

'ref_entries' (sometimes the number after FIGREF is off like from FIGREF1 to FIGREF3 with no FIGREF2 and also may have TABREF)
- 'FIGREF0'
  - 'text' (to be extracted)
  - 'latex'
  - 'type'
- 'FIGREF1'
- 'FIGREF (whatever number)'
- 'TABREF0'
  - 'text' (to be extracted)
  - 'latex'
  - 'type'
- 'TABREF1'
- 'TABREF (whatever number)'

'back_matter'
- 0
  - 'cite_spans'
    - 0
      - 'start'
      - 'end'
      - 'text' (not relevant)
      - 'ref_id'
    - 1
    - (whatever number)
  - 'ref_spans'
    - 0
      - 'start'
      - 'end'
      - 'text' (not relevant)
      - 'ref_id'
    - 1
    - (whatever number)
  - 'section'
  - 'text' (not relevant)
- 1
- (whatever number)